In [1]:
import pandas as pd

In [2]:
subway=pd.read_parquet('subway_final3.parquet')

In [4]:
subway.head()
subway.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9728415 entries, 0 to 9769202
Data columns (total 4 columns):
 #   Column             Dtype         
---  ------             -----         
 0   transit_timestamp  datetime64[ns]
 1   ridership          int16         
 2   latitude           float64       
 3   longitude          float64       
dtypes: datetime64[ns](1), float64(2), int16(1)
memory usage: 315.4 MB


In [5]:
taxi=pd.read_parquet('c:/fomo_data/new_taxi20221.parquet')

In [10]:
taxi.head()

,LocationID,datetime_rounded,passenger_count
0,2,2022-02-06 15:00:00,1.0
1,2,2022-02-16 23:00:00,1.0
2,2,2022-02-23 08:30:00,1.0
3,2,2022-02-23 19:30:00,4.0
4,2,2022-03-22 10:30:00,1.0


In [8]:
taxi2=pd.read_parquet('c:/fomo_data/new_taxi20222.parquet')

In [9]:
taxi2.head()

,LocationID,datetime_rounded,passenger_count
0,2,2022-07-28 15:30:00,1.0
1,2,2022-08-16 00:00:00,1.0
2,2,2022-08-17 23:30:00,1.0
3,2,2022-08-19 15:30:00,2.0
4,2,2022-08-20 14:00:00,1.0


In [11]:
taxi = pd.concat([taxi,taxi2], ignore_index=True)

In [12]:
taxi['datetime_rounded'] = taxi['datetime_rounded'].dt.floor('H')

C:\Users\Kevin\AppData\Local\Temp\ipykernel_13792\588519641.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  taxi['datetime_rounded'] = taxi['datetime_rounded'].dt.floor('H')


In [14]:
import geopandas as gpd

#sahpe file for the taxi zones
taxi_zones = gpd.read_file('c:/fomo_data/taxi.shp')

# Convert subway latitude and longitude to a GeoDataFrame
geometry = gpd.points_from_xy(subway['longitude'], subway['latitude'])
subway_geo = gpd.GeoDataFrame(subway, geometry=geometry, crs="EPSG:4326")

# Reproject taxi_zones to the same CRS as subway_geo if needed
taxi_zones = taxi_zones.to_crs(subway_geo.crs)

# Perform a spatial join to find the taxi zone for each subway station using 'predicate' instead of 'op'
subway_geo = gpd.sjoin(subway_geo, taxi_zones, how='left', predicate='intersects')

In [15]:
subway_geo.head()

,transit_timestamp,ridership,latitude,longitude,geometry,index_right,borough,location_i,objectid,shape_area,shape_leng,zone
0,2023-01-01 01:00:00,52,40.775036,-73.912034,POINT (-73.91203 40.77504),225,Queens,223.0,223.0,0.000764,0.166022,Steinway
1,2023-01-01 01:00:00,95,40.759901,-73.984139,POINT (-73.98414 40.75990),229,Manhattan,230.0,230.0,0.000056,0.031028,Times Sq/Theatre District
2,2023-01-01 01:00:00,52,40.706870,-73.953431,POINT (-73.95343 40.70687),216,Brooklyn,217.0,217.0,0.000115,0.055391,South Williamsburg
3,2023-01-01 01:00:00,249,40.708359,-73.957757,POINT (-73.95776 40.70836),216,Brooklyn,217.0,217.0,0.000115,0.055391,South Williamsburg
4,2023-01-01 01:00:00,394,40.720280,-73.993915,POINT (-73.99392 40.72028),151,Manhattan,148.0,148.0,0.000070,0.039131,Lower East Side


In [17]:
# taxi zone ids for eaahc of the subway stations
subway['locationid'] = subway_geo['location_i']

In [18]:
subway.head()

,transit_timestamp,ridership,latitude,longitude,locationid
0,2023-01-01 01:00:00,52,40.775036,-73.912034,223.0
1,2023-01-01 01:00:00,95,40.759901,-73.984139,230.0
2,2023-01-01 01:00:00,52,40.706870,-73.953431,217.0
3,2023-01-01 01:00:00,249,40.708359,-73.957757,217.0
4,2023-01-01 01:00:00,394,40.720280,-73.993915,148.0


In [19]:
taxi.head()

,LocationID,datetime_rounded,passenger_count
0,2,2022-02-06 15:00:00,1.0
1,2,2022-02-16 23:00:00,1.0
2,2,2022-02-23 08:00:00,1.0
3,2,2022-02-23 19:00:00,4.0
4,2,2022-03-22 10:00:00,1.0


In [20]:
subway.to_parquet('c:/fomo_data/subway_taxi_id.parquet')

In [23]:
subway['locationid'] = subway['locationid'].astype('category')

In [24]:
subway.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9728415 entries, 0 to 9769202
Data columns (total 5 columns):
 #   Column             Dtype         
---  ------             -----         
 0   transit_timestamp  datetime64[ns]
 1   ridership          int16         
 2   latitude           float64       
 3   longitude          float64       
 4   locationid         category      
dtypes: category(1), datetime64[ns](1), float64(2), int16(1)
memory usage: 334.0 MB


In [25]:
subway.drop(columns=['longitude', 'latitude'], inplace=True)

In [26]:
subway.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9728415 entries, 0 to 9769202
Data columns (total 3 columns):
 #   Column             Dtype         
---  ------             -----         
 0   transit_timestamp  datetime64[ns]
 1   ridership          int16         
 2   locationid         category      
dtypes: category(1), datetime64[ns](1), int16(1)
memory usage: 185.6 MB


In [27]:
taxi.rename(columns={
    'LocationID': 'locationid',
    'datetime_rounded': 'transit_timestamp',
    'passenger_count': 'ridership'
}, inplace=True)

In [28]:
df = pd.concat([subway, taxi], ignore_index=True)

In [29]:
df = df.groupby(['locationid', 'transit_timestamp']).agg({'ridership': 'sum'}).reset_index()

In [30]:
print(len(df))

3527539


In [31]:
df.to_parquet('c:/fomo_data/sub_taxi_combine.parquet')

In [33]:
df['locationid'].nunique()

260